In [1]:
import torch

In [2]:
import os

In [5]:
import os

# Path to your image folder
folder_path = "../"

# Get all files in the folder
files = os.listdir(folder_path)

# Filter only image files (you can add more extensions if needed)
image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

# Sort (optional, for consistent order)
image_files.sort()

# Loop through and rename
for i, filename in enumerate(image_files, start=1):
    # Get the file extension
    print(i)
    ext = os.path.splitext(filename)[1]
    
    # Create new filename like image_1.jpg
    new_name = f"image_{i}{ext}"
    
    # Old and new paths
    old_path = os.path.join(folder_path, filename)
    new_path = os.path.join(folder_path, new_name)
    
    # Rename the file
    os.rename(old_path, new_path)

print("✅ All images renamed successfully!")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
import cv2
input_dir='../'
output_dir='../cropped'

os.makedirs(output_dir,exist_ok=True)

target_size =(512,512)

for file in os.listdir(input_dir):
    if file.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff')):
        img_path=os.path.join(input_dir,file)
        img=cv2.imread(img_path)
        print(img_path)
        resized=cv2.resize(img,target_size)
        cv2.imwrite(os.path.join(output_dir,file),resized)
print('done')

../image_1.jpg
../image_10.jpg
../image_100.jpg
../image_1000.jpg
../image_1001.jpg
../image_1002.jpg
../image_1003.jpg
../image_1004.jpg
../image_1005.jpg
../image_1006.jpg
../image_1007.jpg
../image_1008.jpg
../image_1009.jpg
../image_101.jpg
../image_1010.jpg
../image_1011.jpg
../image_1012.jpg
../image_1013.jpg
../image_1014.jpg
../image_1015.jpg
../image_1016.jpg
../image_1017.jpg
../image_1018.jpg
../image_1019.jpg
../image_102.jpg
../image_1020.jpg
../image_1021.jpg
../image_1022.jpg
../image_1023.jpg
../image_1024.jpg
../image_1025.jpg
../image_1026.jpg
../image_1027.jpg
../image_1028.jpg
../image_1029.jpg
../image_103.jpg
../image_1030.jpg
../image_1031.jpg
../image_1032.jpg
../image_1033.jpg
../image_1034.jpg
../image_1035.jpg
../image_1036.jpg
../image_1037.jpg
../image_1038.jpg
../image_1039.jpg
../image_104.jpg
../image_1040.jpg
../image_1041.jpg
../image_1042.jpg
../image_1043.jpg
../image_1044.jpg
../image_1045.jpg
../image_1046.jpg
../image_1047.jpg
../image_1048.jpg
..

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.5/39.0 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.5/39.0 MB 1.7 MB/s eta 0:00:23
   - -------------------------------------- 1.0/39.0 MB 1.1 MB/s eta 0:00:36
   - -------------------------------------- 1.0/39.0 MB 1.1 MB/s eta 0:00:36
   - -------------------------------------- 1.0/39.0 MB 1.1 MB/s eta 0:00:36
   - -------------------------------------- 1.3/39.0 MB 791.3 kB/s eta 0:00:48
   - -------------------------------------- 1.3/39.0 MB 791.3 kB/s eta 0:00:48
   - -------------------------------------- 1.3/39.0 MB 791.3 kB/s eta 0:00:48
   - -------------------------------------- 1.6/39.0 MB 674.9 kB/s eta 0:00:56
   -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.


In [4]:
import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from numpy import linalg as LA
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from tqdm import tqdm

In [6]:
class DoubleConv(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.conv_op=nn.Sequential(
            nn.Conv2d(in_chan,out_chan,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_chan,out_chan,kernel_size=3,padding=1),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.conv_op(x)

In [7]:
class Downsampler(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.conv=DoubleConv(in_chan,out_chan)
        self.poll=nn.MaxPool2d(kernel_size=2,stride=2)
    def forward(self,x):
        down=self.conv(x)
        p=self.pool(down)
        return down , p

In [8]:
class Upsampler(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.up=nn.ConvTranspose2d(in_chan,in_chan//2,kernel_size=2,stride=2)
        self.conv=DoubleConv(in_chan , out_chan)
    def forward(self,x1,x2):
        x1=self.up(x1)
        x=torch.cat([x1,x2],1)
        return self.conv(x)
    

In [ ]:
class UNet(nn.Module):
    def __init__(self,in_chan,num_classes):
        super().__init__()
        self.down_c1=Downsampler(in_channels,64)
        self.down_c2=Downsampler(64,128)
        self.down_c3=Downsampler(128,256)
        self.down_c4=Downsampler(256,512)

        self.bottle_neck=DoubleConv(512,1024)

        self.up_c1=Upsampler(1024,512)
        self.up_c2=Upsampler(512,256)
        self.up_c3=Upsampler(256,128)
        self.up_c4=Upsampler(128,64)

        self.out=nn.Conv2d(in_chan=64,out_)

In [1]:
import numpy as np
import pandas as pd
import torch 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder


In [6]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv')

In [7]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [9]:
df.drop(columns={'id','Unnamed: 32'},inplace=True)

In [10]:
X_train ,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [11]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [12]:
X_train

array([[-1.24454464, -0.90604522, -1.16008058, ...,  0.59120777,
         0.27818319,  2.3116552 ],
       [ 1.68161255, -1.13777425,  1.70879369, ...,  0.67265669,
        -0.8998767 , -0.405955  ],
       [ 1.04422629, -2.0554212 ,  1.21402149, ...,  2.22471119,
         2.64222005,  1.92437361],
       ...,
       [ 1.32966449,  1.23744829,  1.18184119, ...,  1.17190843,
         1.62091813,  0.48479264],
       [-0.68225911,  0.54457849, -0.7200149 , ..., -1.21801416,
        -0.79062115, -1.02283924],
       [-1.08963207,  1.93726995, -1.08405461, ..., -0.40171494,
        -1.11838781, -0.07123306]], shape=(455, 30))

In [13]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [14]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [35]:
class neuralnet():
    def __init__ (self,X):
        self.weight=torch.rand(X.shape[1],1,dtype=torch.float64 , requires_grad=True)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)
        
    def forward(self,X):
        z=torch.matmul(X , self.weight) +self.bias
        y_pred=torch.sigmoid(z)
        return y_pred
    def loss_function(self,y_pred,y):
        epsilon=1e-7
        y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
        loss=-(y*torch.log(y_pred)+(1-y_train_tensor)*torch.log(1-y_pred)).mean()
        return loss

In [45]:
lr=0.1
epochs=100

In [37]:
model=neuralnet(X_train_tensor)


In [38]:
model.bias

tensor([0.], dtype=torch.float64, requires_grad=True)

In [46]:
# training

#create a model 

#loop

for epoch in range(epochs):
    
    
# forward pass
    y_pred=model.forward(X_train_tensor)
#loss calculate
    loss=model.loss_function(y_pred,y_train_tensor)
    print(epoch+1, loss.item())
#backward pass
    loss.backward()
    with torch.no_grad():
        model.weight-=lr*model.weight.grad
        model.bias-= lr*model.bias.grad
    model.weight.grad.zero_()
    model.bias.grad.zero_()


1 0.6806056133231201
2 0.6805405141100758
3 0.6804764403026737
4 0.6804133658863051
5 0.6803512657846567
6 0.6802901158141322
7 0.6802298926409186
8 0.6801705737405345
9 0.6801121373597041
10 0.6800545624804153
11 0.679997828786025
12 0.6799419166292839
13 0.6798868070021639
14 0.6798324815073733
15 0.6797789223314612
16 0.6797261122194059
17 0.6796740344506011
18 0.6796226728161534
19 0.6795720115974075
20 0.6795220355456275
21 0.6794727298627625
22 0.6794240801832301
23 0.6793760725566552
24 0.6793286934315097
25 0.6792819296395938
26 0.6792357683813133
27 0.6791901972117004
28 0.6791452040271393
29 0.6791007770527475
30 0.6790569048303826
31 0.6790135762072289
32 0.6789707803249375
33 0.6789285066092832
34 0.6788867447603074
35 0.678845484742925
36 0.6788047167779613
37 0.678764431333598
38 0.6787246191172069
39 0.6786852710675457
40 0.6786463783472978
41 0.678607932335936
42 0.6785699246228953
43 0.6785323470010317
44 0.6784951914603581
45 0.6784584501820391
46 0.6784221155326301
4

In [1]:
import torch.nn as nn    
from torch.utils.data import DataLoader , Dataset

In [ ]:
class imgdata(Dataset):
    def __init__(self):
        super().__init__()
        self.

In [2]:
class CNNBlock(nn.Module):
    def __init__(self,in_chan,out_chan) -> None:
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(in_chan,out_chan,4,stride=2,bias=False,padding_mode='reflect'),
            nn.BatchNorm2d(out_chan),
            nn.LeakyReLU(0.2)
        )
    def forward(self,x):
        return self.conv(x)
    

In [ ]:
class Descriminator(nn.Module):
    def __init__(self,in_chan,features) -> None:
        super().__init__()
        
        self.initial=nn.Sequential(
            nn.Conv2d(in_chan*2,features,kernel_size=4,stride=2,padding=1),
            nn.LeakyReLU(0.2)
        )
        layers=[]
        

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os


class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=3):
        super(UNetGenerator, self).__init__()
        

        self.enc1 = self.conv_block(in_channels, 64, normalize=False)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.enc5 = self.conv_block(512, 512)
        self.enc6 = self.conv_block(512, 512)
        self.enc7 = self.conv_block(512, 512)

        self.bottleneck = nn.Sequential(
            nn.Conv2d(512, 512, 4, 2, 1),
            nn.ReLU(inplace=True)
        )
        

        self.dec1 = self.deconv_block(512, 512, dropout=True)
        self.dec2 = self.deconv_block(1024, 512, dropout=True)
        self.dec3 = self.deconv_block(1024, 512, dropout=True)
        self.dec4 = self.deconv_block(1024, 512)
        self.dec5 = self.deconv_block(1024, 256)
        self.dec6 = self.deconv_block(512, 128)
        self.dec7 = self.deconv_block(256, 64)
        
        # Final output layer
        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )
    
    def conv_block(self, in_channels, out_channels, normalize=True):
        layers = [nn.Conv2d(in_channels, out_channels, 4, 2, 1)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return nn.Sequential(*layers)
    
    def deconv_block(self, in_channels, out_channels, dropout=False):
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        if dropout:
            layers.append(nn.Dropout(0.5))
        return nn.Sequential(*layers)
    
    def forward(self, x):
  
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        e5 = self.enc5(e4)
        e6 = self.enc6(e5)
        e7 = self.enc7(e6)
        
  
        b = self.bottleneck(e7)
        

        d1 = self.dec1(b)
        d1 = torch.cat([d1, e7], dim=1)
        
        d2 = self.dec2(d1)
        d2 = torch.cat([d2, e6], dim=1)
        
        d3 = self.dec3(d2)
        d3 = torch.cat([d3, e5], dim=1)
        
        d4 = self.dec4(d3)
        d4 = torch.cat([d4, e4], dim=1)
        
        d5 = self.dec5(d4)
        d5 = torch.cat([d5, e3], dim=1)
        
        d6 = self.dec6(d5)
        d6 = torch.cat([d6, e2], dim=1)
        
        d7 = self.dec7(d6)
        d7 = torch.cat([d7, e1], dim=1)
        
        return self.final(d7)


class PatchGANDiscriminator(nn.Module):
    def __init__(self, in_channels=4):
        super(PatchGANDiscriminator, self).__init__()
        
        self.model = nn.Sequential(
            # Input: grayscale (1) + color (3) = 4 channels
            nn.Conv2d(in_channels, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, 4, 1, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            # PatchGAN output
            nn.Conv2d(512, 1, 4, 1, 1)
        )
    
    def forward(self, grayscale, color):
        x = torch.cat([grayscale, color], dim=1)
        return self.model(x)


# Dataset class
class ColorizationDataset(Dataset):
    def __init__(self, image_dir, transform=None, img_size=256):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) 
                           if f.endswith(('.jpg', '.jpeg', '.png'))]
        self.img_size = img_size
        self.transform = transform or transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
        ])
        self.color_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        color_img = Image.open(img_path).convert('RGB')
        gray_img = color_img.convert('L')
        
        gray_tensor = self.transform(gray_img)
        color_tensor = self.color_transform(color_img)
        
        return gray_tensor, color_tensor



def train_pix2pix(generator, discriminator, dataloader, num_epochs=100, 
                  device='cuda', lambda_l1=100, lr=0.0002):
    

    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    

    criterion_GAN = nn.BCEWithLogitsLoss()
    criterion_L1 = nn.L1Loss()
    

    generator = generator.to(device)
    discriminator = discriminator.to(device)
    

    for epoch in range(num_epochs):
        for i, (gray_imgs, color_imgs) in enumerate(dataloader):
            batch_size = gray_imgs.size(0)
            gray_imgs = gray_imgs.to(device)
            color_imgs = color_imgs.to(device)
            

            real_labels = torch.ones(batch_size, 1, 30, 30).to(device)
            fake_labels = torch.zeros(batch_size, 1, 30, 30).to(device)
            

            optimizer_D.zero_grad()

            real_output = discriminator(gray_imgs, color_imgs)
            d_real_loss = criterion_GAN(real_output, real_labels)
            

            fake_color = generator(gray_imgs)
            fake_output = discriminator(gray_imgs, fake_color.detach())
            d_fake_loss = criterion_GAN(fake_output, fake_labels)

            d_loss = (d_real_loss + d_fake_loss) * 0.5
            d_loss.backward()
            optimizer_D.step()
            

            optimizer_G.zero_grad()
            

            fake_color = generator(gray_imgs)
            fake_output = discriminator(gray_imgs, fake_color)
            

            g_gan_loss = criterion_GAN(fake_output, real_labels)
            

            g_l1_loss = criterion_L1(fake_color, color_imgs)
  
            g_loss = g_gan_loss + lambda_l1 * g_l1_loss
            g_loss.backward()
            optimizer_G.step()

            if i % 50 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'D_loss: {d_loss.item():.4f} G_loss: {g_loss.item():.4f} '
                      f'G_GAN: {g_gan_loss.item():.4f} G_L1: {g_l1_loss.item():.4f}')

        if (epoch + 1) % 10 == 0:
            torch.save({
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
            }, f'pix2pix_checkpoint_epoch_{epoch+1}.pth')
            print(f'Checkpoint saved at epoch {epoch+1}')



def colorize_image(generator, gray_image_path, device='cuda'):
    generator.eval()
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    
    gray_img = Image.open(gray_image_path).convert('L')
    gray_tensor = transform(gray_img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        colorized = generator(gray_tensor)
    
  
    colorized = colorized.squeeze(0).cpu() * 0.5 + 0.5
    colorized = colorized.clamp(0, 1)
    
    return transforms.ToPILImage()(colorized)



In [5]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp313-cp313-win_amd64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:

if __name__ == '__main__':

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    

    generator = UNetGenerator(in_channels=1, out_channels=3)
    discriminator = PatchGANDiscriminator(in_channels=4)
    

    dataset = ColorizationDataset('../cropped', img_size=256)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)
    

    train_pix2pix(
        generator=generator,
        discriminator=discriminator,
        dataloader=dataloader,
        num_epochs=100,
        device=device,
        lambda_l1=100,
        lr=0.0002
    )
    


Using device: cuda
